## Homework

### Dataset


In this homework, we will use the California Housing Prices data from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

Here's a wget-able [link](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv):

**Dataset saved into the folder data**

### Features

For the rest of the homework, you'll need to use only these columns:

* `'latitude'`,
* `'longitude'`,
* `'housing_median_age'`,
* `'total_rooms'`,
* `'total_bedrooms'`,
* `'population'`,
* `'households'`,
* `'median_income'`,
* `'median_house_value'`
* `'ocean_proximity'`,

In [1]:
features = ['latitude',
            'longitude',
            'housing_median_age',
            'total_rooms',
            'total_bedrooms',
            'population',
            'households',
            'median_income',
            'median_house_value',
            'ocean_proximity']

### Data preparation

* Select only the features from above and fill in the missing values with 0.
* Create a new column `rooms_per_household` by dividing the column `total_rooms` by the column `households` from dataframe. 
* Create a new column `bedrooms_per_room` by dividing the column `total_bedrooms` by the column `total_rooms` from dataframe. 
* Create a new column `population_per_household` by dividing the column `population` by the column `households` from dataframe. 

In [2]:
import pandas as pd
import numpy as np
import random

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('data/housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
len(df)

20640

In [5]:
data = df[features].copy()
data.isna().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [6]:
data['total_bedrooms'] = data.total_bedrooms.fillna(0)
data['rooms_per_household'] = data.total_rooms / data.households
data['bedrooms_per_room'] = data.total_bedrooms / data.total_rooms
data['population_per_household'] = data.population / data.households
data['bedrooms_per_room'] = data.bedrooms_per_room.fillna(0)

### Question 1

What is the most frequent observation (mode) for the column `ocean_proximity`?

Options:
* `NEAR BAY`
* `<1H OCEAN`
* `INLAND`
* `NEAR OCEAN`

In [7]:
data.ocean_proximity.mode().values[0]

'<1H OCEAN'

**Answer: <1H OCEAN**

## Split the data



* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to 42.
* Make sure that the target value (`median_house_value`) is not in your dataframe.

In [8]:
SEED = 42

In [9]:
random.seed(SEED)
np.random.seed(SEED)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df_train_full, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.2, random_state=42)

In [12]:
y_train = df_train.median_house_value.values
y_val = df_val.median_house_value.values
y_test = df_test.median_house_value.values

In [13]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

### Question 2

* Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your train dataset.
    - In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?

Options:
* `total_bedrooms` and `households`
* `total_bedrooms` and `total_rooms`
* `population` and `households`
* `population_per_household` and `total_rooms`

In [14]:
numeric_features = np.delete(df_train.columns.values, 8)
cat_features = df_train.columns.values[8]

In [15]:
corr_mat = df_train[numeric_features].corr()
abs_corr_mat = corr_mat[corr_mat != 1].abs()

In [16]:
max_corr_col = abs_corr_mat[abs_corr_mat.values == abs_corr_mat.max().max()].index

In [17]:
print(f'Max correlation between {max_corr_col[0]} and {max_corr_col[1]}')

Max correlation between total_bedrooms and households


**Answer: total_bedrooms and households**

### Make `median_house_value` binary

* We need to turn the `median_house_value` variable from numeric into binary.
* Let's create a variable `above_average` which is `1` if the `median_house_value` is above its mean value and `0` otherwise.

In [18]:
avg_price = df.median_house_value.mean()

In [19]:
above_average_train = np.array(y_train > avg_price, dtype=int)

### Question 3

* Calculate the mutual information score with the (binarized) price for the categorical variable that we have. Use the training set only.
* What is the value of mutual information?
* Round it to 2 decimal digits using `round(score, 2)`

Options:
- 0.263
- 0.00001
- 0.101
- 0.15555

In [20]:
from sklearn.metrics import mutual_info_score

In [21]:
round(mutual_info_score(above_average_train, df_train[cat_features]), 2)

0.1

**Answer: 0.101**

### Question 4

* Now let's train a logistic regression
* Remember that we have one categorical variable `ocean_proximity` in the data. Include it using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

Options:
- 0.60
- 0.72
- 0.84
- 0.95

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [23]:
encoder = OneHotEncoder().fit(df_train[cat_features].values.reshape(-1,1))

In [24]:
one_hot_features = ['INLAND', '1H_OCEAN', 'NEAR_OCEAN', 'NEAR_BAY', 'ISLAND']

In [25]:
df_train[one_hot_features] = encoder.transform(df_train[cat_features].values.reshape(-1,1)).toarray()
df_val[one_hot_features] = encoder.transform(df_val[cat_features].values.reshape(-1,1)).toarray()
df_test[one_hot_features] = encoder.transform(df_test[cat_features].values.reshape(-1,1)).toarray()

In [26]:
del df_train['ocean_proximity']
del df_val['ocean_proximity']
del df_test['ocean_proximity']

In [27]:
y_train_b = above_average_train
y_val_b = np.array(y_val > avg_price, dtype=int)
y_test_b = np.array(y_test > avg_price, dtype=int)

In [28]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(df_train, y_train_b)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [29]:
val_acc = round(accuracy_score(y_val_b, model.predict(df_val)), 2)
val_acc

0.83

**Answer: 0.84**

### Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 
* Which of following feature has the smallest difference? 
   * `total_rooms`
   * `total_bedrooms` 
   * `population`
   * `households`

> **note**: the difference doesn't have to be positive

In [30]:
from sklearn.feature_selection import RFE

In [31]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
selector = RFE(model).fit(df_train, y_train_b)

In [32]:
# the least useful features
least_useful = df_train.columns.values[selector.support_ == False]

In [33]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(df_train, y_train_b)
val_acc = accuracy_score(y_val_b, model.predict(df_val))
val_acc

0.8280351195882532

In [34]:
new_features = df_train.columns.values
diff_list = {}
for feature in least_useful:
    train_features = new_features[new_features != feature]
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(df_train[train_features], y_train_b)
    new_val_acc = accuracy_score(y_val_b, model.predict(df_val[train_features]))
    diff_list[feature] = abs(val_acc - new_val_acc)

In [35]:
diff_list

{'latitude': 0.0018165304268846771,
 'longitude': 0.0,
 'housing_median_age': 0.004541326067211693,
 'total_rooms': 0.0006055101422948184,
 'total_bedrooms': 0.001513775355737157,
 'population': 0.008174386920980936,
 'households': 0.0036330608537693543,
 'rooms_per_household': 0.0009082652134422275}

**Answer: total_bedrooms**

### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column `'median_house_value'`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model (`model = Ridge(alpha=a, solver="sag", random_state=42)`) on the training data.
* This model has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

Options:
- 0
- 0.01
- 0.1
- 1
- 10

In [36]:
from sklearn.linear_model import Ridge

In [37]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [38]:
y_train = np.log1p(y_train)
y_val = np.log1p(y_val)
y_test = np.log1p(y_test)

In [39]:
alpha_ls = [0, 0.01, 0.1, 1, 10]
scores = []
for a in alpha_ls:
    model = Ridge(alpha=a, solver="sag", random_state=42).fit(df_train, y_train)
    score = rmse(y_val, model.predict(df_val))
    scores.append(round(score))
alpha_ls[scores.index(min(scores))]

0

In [40]:
scores

[0.5234968906587238,
 0.5234968906810293,
 0.5234968908817702,
 0.523496892872448,
 0.5234969128126647]

**Answer: 0**